<h1><center>10.BACKGROUND SUBTRACTION, OBJECT TRACKING AND SCENE CHANGE DETECTION</center></h1>

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

<h1>BackGround Subtraction for Outdoor Video</h1>

In [ ]:
video = cv2.VideoCapture(r"video.mp4") #Replace with your Video path

ret, background = video.read()
if not ret:
    print("Error: Unable to read video")
    exit()

#To Save Output
output_path = r"Output\outputvideo.mp4"  #Replace with your Video Output path
frame_width = 1080
frame_height = 720
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(video.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width,frame_height),isColor=False)

#To clean noise
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=80,detectShadows=True)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(6,6)) 

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    frame = cv2.resize(frame, (frame_width, frame_height))

    fg_mask = fgbg.apply(frame)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
    
    cv2.imshow('Original Frame', frame)
    cv2.imshow('Foreground Mask', fg_mask)
    out.write(fg_mask)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
out.release()
cv2.destroyAllWindows()

#print(f"Output path : {output_path}")

<h1>BackGround Subtraction For Indoor Video</h1>

In [ ]:
video = cv2.VideoCapture(r"indoor.mp4") #Replace with your Video  path

ret, background = video.read()
if not ret:
    print("Error: Unable to read video")
    exit()

#To Save Output
output_path = r"Output\outputindoor.mp4" #Replace with your Video Output path
frame_width = 720
frame_height = 720
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(video.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width,frame_height),isColor=False)

#To clean noise 
fgbg = cv2.createBackgroundSubtractorMOG2()
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)) 

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    frame = cv2.resize(frame, (frame_width, frame_height))

    fg_mask = fgbg.apply(frame)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
    
    cv2.imshow('Original Frame', frame)
    cv2.imshow('Foreground Mask', fg_mask)
    out.write(fg_mask)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
out.release()
cv2.destroyAllWindows()
#print(f"Output path : {output_path}")

<h2>Object Tracking</h2>

In [ ]:
video_path = r"ob_track.mp4" #Replace with your Video  path
cap = cv2.VideoCapture(video_path)

#background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

output_path = r"Output\object_tracking.mp4" #Replace with your Video Output path
frame_width = 720
frame_height = 800
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(video.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width,frame_height),isColor=False)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame,(frame_width, frame_height))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Applying background subtraction
    fgmask = fgbg.apply(gray)
    
    # Removing noise using morphology
    kernel = np.ones((5, 5), np.uint8)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    
    # Finding contours (detected objects)
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Drawing bounding boxes around detected objects
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Filter small objects
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    cv2.imshow('Object Detection and Tracking', frame)
    out.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 

cap.release()
out.release()
cv2.destroyAllWindows()
#print(f"Output path : {output_path}")

<h2>Scene Change Detection</h2>

In [ ]:
video_path = r"scene_ch.mp4" #Replace with your Video  path
cap = cv2.VideoCapture(video_path)

#background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

#Parameters for scene change
prev_frame = None
scene_change_threshold = 50.0
count=0

#Save Output
output_path = r"scene_change.mp4" #Replace with your Video Output path
frame_width = 1080
frame_height = 720
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(video.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width,frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break


    frame = cv2.resize(frame, (frame_width, frame_height))
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect scene change
    if prev_frame is not None:
        diff = cv2.absdiff(prev_frame, gray)
        non_zero_count = np.count_nonzero(diff)
        if non_zero_count > scene_change_threshold:
            count=count+1
    prev_frame = gray
    
    # Applying background subtraction
    fgmask = fgbg.apply(gray)
    
    # Removing noise using morphology
    kernel = np.ones((5, 5), np.uint8)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    
    # Finding contours (detected objects)
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Drawing bounding boxes around detected objects
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Filter small objects
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
    cv2.imshow('Object Detection and Tracking', frame)
    out.write(frame)
    if cv2.waitKey(30) & 0xFF == 27:
        break

print(f"Detected Screen Changes :{count}") 
cap.release()
out.release()
cv2.destroyAllWindows()
#print(f"Output path : {output_path}")

Detected Screen Changes :269
